Notebook com exemplo de algumas funcionalidades que o projeto pretende implementar (em andamento....)

Coleta dos tweets com tweepy

In [7]:
import tweepy
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [8]:
# Para coletar tweets via API do twitter e necessario ter uma conta de desenvolvedor na plataforma
# Esse arquivo .txt constam minhas chaves e senhas pra acesso, nao disponibilizados no repositorio,
# respeitando os termos de uso do twitter e garantindo a privacidade da minha conta

with open('./twitter_keys.txt', 'r') as f:
    api_key = f.readline().strip('\n')
    api_key_secret = f.readline().strip('\n')
    acess_token = f.readline().strip('\n')
    acess_token_secret = f.readline().strip('\n')


auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(acess_token, acess_token_secret)

api = tweepy.API(auth)

In [9]:
'''
palavras_chave_hashtags = input(
    'Insira aqui as palavras chaves e/ou hashtags relacionadas ao tema de interesse separados por espaços:'
    )
'''

palavras_chave_hashtags = 'lula alckmin' # configurando palavras chaves do exemplo

In [16]:
lista_query = []
for palavra in palavras_chave_hashtags.split():
    query_string = palavra + ' -filter:retweets' # excluindo rts da coleta porque sao textos repetidos
    lista_query.append(query_string)

In [140]:
'''
Transformando json dos tweets em dicts e depois em dataframes e selecionando atributos de interesse para analise
'''


dict_features_ = {}
n = 0

limite = 1_000 # limitando quantidade para nao sobrecarregar as buscas
qtd_por_palavra = limite/len(lista_query)

for query_string in lista_query:

    print(f'Coletando tweets com o termo: {query_string.split()[0]}\n')

    tweets = tweepy.Cursor(api.search_tweets, q=query_string).items(qtd_por_palavra)
    
    for twt in tqdm(tweets, total=int(qtd_por_palavra)):
        d = twt._json
        dict_features_[n] = {}

        dict_features_[n]['created_at'] = d['created_at']
        dict_features_[n]['hashtags'] = d['entities']['hashtags']#['text']
        dict_features_[n]['text'] = d['text']
        dict_features_[n]['tweet_id'] = d['id']
        dict_features_[n]['user_description'] = d['user']['description']
        dict_features_[n]['user_id'] = d['user']['id']
        dict_features_[n]['user_name'] = d['user']['name']
        dict_features_[n]['user_@'] = d['user']['screen_name']


        dict_features_[n]['reply_tweet_id'] = d['in_reply_to_status_id']
        dict_features_[n]['reply_user_@'] = d['in_reply_to_screen_name']
        dict_features_[n]['reply_user_id'] = d['in_reply_to_user_id']

        n+=1

Coletando tweets com o termo: lula



100%|██████████| 500/500 [00:20<00:00, 24.93it/s]


Coletando tweets com o termo: alckmin



100%|██████████| 500/500 [00:19<00:00, 26.04it/s]


In [141]:
df_tweets = pd.DataFrame.from_dict(dict_features_, orient='index')#.to_csv(f'./tweets_lula_alckmin.csv', index=False)
 # selecionando principais features para usar na analise
df_tweets = df_tweets[['text', 'hashtags', 'user_description', 'user_name']]
df_tweets[['text', 'hashtags', 'user_description']].head(10) # omitindo os nomes por questoes de privacidade

,text,hashtags,user_description
0,A prova de que não importa o grau de instrução...,[],🐷
1,"As pessoas estão se perguntando ""Quantos votos...",[],Paixão demais cega
2,A estratégia de Bolsonaro e a Queda de Lula ht...,[],A esquerda é o próprio mal na terra! Com corag...
3,@raismach @andrecdforyou @g1 Manda o Lula para...,[],Ainda q eu ande pelo vale da sombra da morte n...
4,@Sidsto @Gerson72340523 @safbf Tudo a ver. Por...,[],
5,@CMonteroOficial Con una visita del Nobel de l...,[],Premio Nacional de Ciencia y Tecnologia. Menci...
6,@correio Alckmin é esperto! Quer ser vice de L...,[],🇧🇷\nFan account\nInSomnia\nLittle Black Stars
7,Eu quero dormir e acordar em 2023 quando o Lul...,[],~ Artist Wannabe ~\n~ Candle Maker ~\n~ Natura...
8,vai pabllo lance o Lula’s PT Remix,[],se eu for eu vou
9,@silva_gustavo @alanmfreitas @jfdemocracia @sa...,[],Ela|Dela ▪︎ Bissexual 🏳️‍🌈 ...


In [147]:
# as hashtags vem como um dict, com string e localizacao da hashtag no texto - precisa ser modificada
df_tweets['hashtags'].iloc[562]

[{'text': 'Lula', 'indices': [46, 51]}]

Pre-Processamento dos textos

In [148]:
# funcoes auxiliares pra coletar hashtags do texto, bio e nome

import re
HASHTAG = re.compile(r'#(\w+)')

def hashtags_to_list(x):
    try:
      if x == []:
        x = np.NaN
      else:
        x = [x[i]['text'] for i in range(len(x))]
    except:
      x = np.NaN
     
    return x

def get_descriptions_hashtags(d):
  if type(d) != str:
    return np.nan
  else:
    list_hashtags = HASHTAG.findall(d)
    if len(list_hashtags) > 0:
      return list_hashtags
    else:
      return np.nan


def get_username_hashtags(u):
  if type(u) != str:
    return np.nan
  else:
    list_hashtags = HASHTAG.findall(u)
    if len(list_hashtags) > 0:
      return list_hashtags
    else:
      return np.nan

In [149]:
# aplicando as funcoes de hashtags nos dataframes com a abodargem funcional (apply + lambda)

df_tweets['hashtags_list'] = df_tweets.apply(lambda x: hashtags_to_list(x['hashtags']), axis=1)
df_tweets['descriptions_hashtags'] = df_tweets.apply(lambda x: get_descriptions_hashtags(x['user_description']), axis=1)
df_tweets['username_hashtags'] = df_tweets.apply(lambda x: get_username_hashtags(x['user_name']), axis=1)

In [117]:
# pacotes de pre-processamento para nlp (nltk)
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('wordnet')
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rodrigoludermir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rodrigoludermir/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [118]:
''''
Importante: Como não domino inteiramente as expressões regulares, algumas dessas expressões são baseadas em códigos de terceiros, 
coletados em foruns como Stack Overflow. Mas, friso, me baseiei somente na parte de regex, que facilita bastante uma busca otimizada em strings. 
A Classe de pre-processamento, no entanto, é de minha autoria.

'''


from tqdm import tqdm # indica o tempo/quantidade para finalizar loop

class Preprocess():
    def __init__(self, lower=True): # lower=True para tornar tokens em letras minusculas
        self.STOPWORDS = set(stopwords.words('portuguese')) # as palavras mais comuns da língua - "eu, a, o, aquele, eles, voce, da ...
        self.pontuations = string.punctuation # lista de pontuações
        self.tokenizer = RegexpTokenizer(r'\w+') # funcao pra tokenizar os textos
        self.stemmer = nltk.PorterStemmer() #funcao pra stemmatizar tokens
        self.lemmatizer = nltk.WordNetLemmatizer() # funcao pra lemmatizar tokens
        self.hashtags = re.compile(r'#[-_.?&~;+=/#0-9A-Za-z]{1,2076}') # regex pra remover hashtags
        self.mentions = re.compile(r'@[-_.?&~;+=/#0-9A-Za-z]{1,2076}') # regex pra remover mentions em tweets
        self.lower = lower


    # removendo stopwords
    # retorna a lista de palavras que nao estão no conjunto de stopwords, ignorando-as
    def cleaning_stopwords(self, text, STOPWORDS):
        return " ".join([word for word in str(text).split() if word not in STOPWORDS])

    # removendo puntuacoes
    def cleaning_punctuations(self, text, pontuacoes):
        translator = str.maketrans('', '', pontuacoes)
        return text.translate(translator)

    # removendo caracteres repetidos
    def cleaning_repeating_char(self, text):
        return re.sub(r'(.)\1+', r'\1', text)

    # removendo email
    def cleaning_email(self, text):
        return re.sub('@[^\s]+', ' ', text)

    # removendo URL's
    def cleaning_URLs(self, text):
        return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)

    # removendo numeros
    def cleaning_numbers(self, text):
        return re.sub('[0-9]+', '', text)

    # Stemming -  reduzir palavras flexionadas
    def stemming_on_text(self, data, stm):
        text = [stm.stem(word) for word in data]
        return data

    #Lemmatizer - agrupar as formas flexionadas de uma palavra para uma unica feature
    def lemmatizer_on_text(self, data, lm):
        text = [lm.lemmatize(word) for word in data]
        return data
                
    # transformando tweet orignal em tweet pre-processado - limpo
    def transform(self, texts):
        #texts: lista de tweets
        tweets_preprocessados = []
        for t in tqdm(range(len(texts)), total=len(texts), position=0):

            if self.lower:
              tweet = texts[t].lower()
            else:
              tweet = texts[t]
            tweet = self.cleaning_stopwords(tweet, self.STOPWORDS)
            tweet = self.cleaning_punctuations(tweet, self.pontuations)
            tweet = self.cleaning_repeating_char(tweet)
            tweet = self.cleaning_email(tweet)
            tweet = self.cleaning_URLs(tweet)
            tweet = self.cleaning_numbers(tweet)
            tweet = self.hashtags.sub(" ", tweet)
            tweet = self.mentions.sub(" ", tweet)
            tweet = self.tokenizer.tokenize(tweet)
            tweet = self.stemming_on_text(tweet, self.stemmer)
            tweet = self.lemmatizer_on_text(tweet, self.lemmatizer)

            tweets_preprocessados.append(tweet)

        return tweets_preprocessados

In [150]:
# preprocessando textos dos tweets e das bios dos usuarios

prep = Preprocess()
df_tweets['prep_text'] = prep.transform(df_tweets['text'].to_list())
df_tweets['pre_description'] = prep.transform(df_tweets['user_description'].to_list())

100%|██████████| 1000/1000 [00:00<00:00, 4664.94it/s]


In [151]:
df_tweets['prep_text']

0      [prova, importa, grau, instrução, ser, maucará...
1      [pesoas, perguntando, quantos, votos, alckmin,...
2      [estratégia, bolsonaro, queda, lula, htpstcoye...
3      [raismach, andrecdforyou, g, manda, lula, rua,...
4      [sidsto, gerson, safbf, tudo, ver, fazer, chap...
                             ...                        
995    [lula, jantar, alckmin, não, importa, pasado, ...
996    [entusiasmo, aliado, encontro, lulalckmin, cli...
997    [esperei, chapa, lula, boulos, recebi, lula, a...
998    [safbf, n, normalizar, detesto, alckmin, gover...
999    [tá, tudo, bem, pros, lulistas, terem, alckmin...
Name: prep_text, Length: 1000, dtype: object

In [152]:
df_tweets['pre_description']

0                                                     []
1                                 [paixão, demais, cega]
2      [esquerda, próprio, mal, tera, coragem, fé, to...
3      [ainda, q, ande, vale, sombra, morte, temerei,...
4                                                     []
                             ...                        
995    [goiás, orgulha, ter, kajuru, senador, saúde, ...
996                                                 [ai]
997                                  [nada, digno, nota]
998    [caipira, paulo, advogada, administrativista, ...
999                                                   []
Name: pre_description, Length: 1000, dtype: object

Explorando os dados textuais

In [153]:
from collections import Counter

tweets_hashtags = []
username_hashtags = []
descritpions_hashtags = []

for hstg in df_tweets['hashtags_list'].dropna():
    if type(hstg) == list:
        tweets_hashtags.extend(hstg)
    else:
        tweets_hashtags.append(hstg)

df_user_info = df_tweets.copy()
for hstg_d in df_user_info['descriptions_hashtags'].dropna():
    if type(hstg_d) == list:
        descritpions_hashtags.extend(hstg_d)
    else:
        descritpions_hashtags.append(hstg_d)
        
for hstg_u in df_user_info['username_hashtags'].dropna():
    if type(hstg_d) == list:
        username_hashtags.extend(hstg_u)
    else:
        username_hashtags.append(hstg_u)
    

tweets_hashtags_ranked = {k:v for k, v in sorted(Counter(tweets_hashtags).items(), key=lambda x:x[1], reverse=True)}
description_hashtags_ranked = {k:v for k, v in sorted(Counter(descritpions_hashtags).items(), key=lambda x:x[1], reverse=True)}
username_hashtags_ranked = {k:v for k, v in sorted(Counter(username_hashtags).items(), key=lambda x:x[1], reverse=True)}

In [154]:
tweets_htsg = pd.DataFrame.from_dict(tweets_hashtags_ranked, orient='index')
tweets_htsg

,0
EJairOuJaEra,4
LulaLadrao,3
LulaNoPrimeiroTurno,3
Lula,3
Brasil,2
CPIdoCirco,2
STFVergonhaMundial,1
SouLulaSouPT,1
LulaEoPToAmorVencera,1
BolsonaroGenocida,1


In [155]:
username_htsg = pd.DataFrame.from_dict(username_hashtags_ranked, orient='index')
username_htsg

,0
ForaBolsonaro,2
LulaInocente,2
ForaBozo,2
Lula2022,2
Lula,1
LulaPresidente,1
アド中村,1
BrasilConsciente,1
Imunizado,1
ForaBolsoMoro,1


In [156]:
bios_htsg = pd.DataFrame.from_dict(description_hashtags_ranked, orient='index')
bios_htsg 

,0
ForaBolsonaro,6
Lula2022,5
SDV,5
forabolsonarogenocida,3
forabolsonaro,3
...,...
antifascista,1
antiracista,1
Bolsonarolixo,1
JeovaVaiTePegar,1


In [134]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_ngrams(corpus, n=None):
    corpus = [' '.join(tweet) for tweet in corpus]   
    vec = CountVectorizer(ngram_range=(1, 3)).fit(corpus) # de 1 a 3 n-grams 
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return [[n_gram, freq] for n_gram, freq in words_freq][:200] # seleciona as 200 palavras mais frequentes

In [157]:
pd.DataFrame(get_top_ngrams(df_tweets['prep_text']), columns=['ngram', 'frequencia'])

,ngram,frequencia
0,lula,614
1,alckmin,414
2,vai,95
3,pra,92
4,vice,86
...,...,...
195,fabiofaria,7
196,saco,7
197,paulo,7
198,eu,7


In [158]:
pd.DataFrame(get_top_ngrams(df_tweets['pre_description']), columns=['ngram', 'frequencia'])

,ngram,frequencia
0,brasil,36
1,esquerda,33
2,cristão,28
3,and,28
4,bolsonaro,28
...,...,...
195,livre,5
196,chamar,5
197,at,5
198,coração,5
